# Ensemble Model 
Classification in 3 parts

## Prepare Input Data

In [1]:
import sys
sys.path.append("..")
import numpy as np
import sklearn as sk
from sklearn.metrics import classification_report
import torch
from torch import nn, cuda
import matplotlib.pyplot as plt
import prepare_data
from prepare_data import DataPreparationContext
from prepare_data import ClassEmbedders
import models
from models import GenericMutliClassifier

In [2]:
data_context = DataPreparationContext(combine_riskiest=16, verbose=True)

test_data = data_context.prepare_from_file("../../symptom_sum_top16/test.pkl", oversample=False)
X_test_np = test_data['X']
y_test_np = test_data['y']

100%|██████████| 2643/2643 [00:01<00:00, 2079.02it/s]

Input File: ../../symptom_sum_top16/test.pkl
 [depression] has 62
 [anxiety depression] has 44
 [anxiety] has 21
 [ptsd] has 7
 [anxiety ptsd depression] has 3
 [anxiety ptsd] has 7
 [ptsd depression] has 2
 [] has 2054



In [3]:
X_val = models.convert_to_tensors(X_test_np).type(torch.float32).to(models.device)
y_val = models.convert_to_tensors(y_test_np).type(torch.LongTensor).to(models.device)

## 1. Healthy or Sick Classifier

In [4]:
hidden_layers = 250
model = models.GenericMutliClassifier(input_features=X_val.shape[1], output_features=8, hidden_units=hidden_layers).to(models.device)
model.load_model("1-healthy-or-not-multiclassifier.pth")
pred_1 = model.evaluate_model(X_val).cpu()

## 2. Comorbid or Not Classifier

In [5]:
hidden_layers = 250
model = models.GenericMutliClassifier(input_features=X_val.shape[1], output_features=8, hidden_units=hidden_layers).to(models.device)
model.load_model("2-comorbid-or-not-multiclassifier.pth")
pred_2 = model.evaluate_model(X_val).cpu()

## 3. Classifying the Comorbid Conditon

In [6]:
hidden_layers = 250
model = models.GenericMutliClassifier(input_features=X_val.shape[1], output_features=8, hidden_units=hidden_layers).to(models.device)
model.load_model("3-comorbid-codition-multiclassifier.pth")
pred_3 = model.evaluate_model(X_val).cpu()

## Evaluate Results

In [15]:
val_pred = []
val_true = []
y_val_cpu = y_val.cpu()
labels = [(' '.join(x) if len(' '.join(x)) != 0 else 'healthy') for x in prepare_data.CLASSES_COMORBID.values()]

for i in range(len(X_test_np)):
    isHealthy = True if pred_1[i] == 0 else False
    isComorbid = True if pred_2[i] == 2 else False
    comorbidDisease = pred_3[i].numpy()
    if (not isHealthy) and isComorbid and comorbidDisease >= 4:
        val_pred.append(comorbidDisease)
        val_true.append(y_val_cpu[i].numpy())

print(classification_report(y_true=val_true, y_pred=val_pred, target_names=labels, labels=[4,5,6,7]))

                         precision    recall  f1-score   support

     anxiety depression       0.17      0.93      0.29        15
        ptsd depression       0.00      0.00      0.00         0
           anxiety ptsd       0.00      0.00      0.00         3
anxiety ptsd depression       0.00      0.00      0.00         2

              micro avg       0.16      0.70      0.25        20
              macro avg       0.04      0.23      0.07        20
           weighted avg       0.13      0.70      0.22        20



c:\Users\nstan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nstan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nstan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita